## Task 2


Prepare $4$ random $4$-qubit quantum states of your choice. 
  
  
Create and train a variational circuit that transforms input states into predefined output states. Namely  
if random state $1$ is provided, it returns state $\left|0011\right>$  
if random state $2$ is provided, it returns state $\left|0101\right>$  
if random state $3$ is provided, it returns state $\left|1010\right>$  
if random state $4$ is provided, it returns state $\left|1100\right>$
What would happen if you provided a different state?


## Solution
1. The chosen random states are $\left|0000\right>$, $\left|0011\right>$, $\left|0111\right>$, $\left|1011\right>$.  
2. Then I use them as the initial state of QNN $U(\theta)$, and apply the built-in entangled layers (consists of rotation gates and CNOT gates) to the initial states, and run the circuit, and get the final output state $U(\theta)\left|0000\right>$.  
3. Repeat Step $2$ for four times, and get the all four output states$U(\theta)\left|0000\right>$, $U(\theta)\left|0011\right>$, $U(\theta)\left|0111\right>$, $U(\theta)\left|1011\right>$.  
4. Calculate the square of inner products between the output states and the target states, and add them together as the loss function, i.e., $\mathcal{L}=\sum\limits_{i=1}^{4}\left|\left<{\psi_i}\right|\left|{\phi_i}\right>\right|^2=\left|\left<0000\right|U(\theta)^\dagger\left|0011\right>\right|^2+\left|\left<0011\right|U(\theta)^\dagger\left|0101\right>\right|^2+\left|\left<0111\right|U(\theta)^\dagger\left|1010\right>\right|^2+\left|\left<1011\right|U(\theta)^\dagger\left|1100\right>\right|^2$  
5. Use Adam optimizer to minimize the loss.  
6. Update the parameters in the QNN.  
7. Repeat Step $2-6$ for $120$ iterations and get the optimal parameters in the QNN.  
   
As you can see, the final loss after training is $-4$, which is desired value, since all inner product are equal to $1$, which means the output states are indeed the target states given the chosen input.     
    
The whole QNN is a unitary, it maps from a set of basis to another set of basis. In this case, it maps $\left|0000\right>$ to $\left|0011\right>$, $\left|0011\right>$ to $\left|0101\right>$, $\left|0111\right>$ to $\left|1010\right>$, and $\left|1011\right>$ to $\left|1100\right>$. It can be seen as a $16\times 16$ matrix, and four columns are determined, while other $12$ columns does not matter and depends on the optimization process. So if superpositions of $\left|0000\right>$, $\left|0011\right>$, $\left|0111\right>$, $\left|1011\right>$ are given, then the output will be the superpositions of the four target states. Otherwise, it will depend on the other $12$ colmns.

In [1]:
import paddle
from paddle_quantum.circuit import UAnsatz

import numpy as np
from numpy import pi as PI

/Users/zihanxia/anaconda3/envs/paddle_quantum/lib/python3.7/site-packages/ipykernel/pylab/backend_inline.py:164: DeprecationWarning: `configure_inline_support` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.configure_inline_support()`
  configure_inline_support(ip, backend)


In [2]:
def U_theta(theta, N, D, initial_state, target_state):
    """
    Quantum Neural Network
    """
    # Initialize the quantum neural network according to the number of qubits N
    cir = UAnsatz(N)
    # Built-in {R_y + CNOT + U3} circuit template
    cir.complex_entangled_layer(theta[:D], D)
    # Lay R_y gates in the last row
    for i in range(N):
        cir.ry(theta=theta[D][i][0], which_qubit=i)
    # The quantum neural network acts on one of the random initial states
    fin_state = cir.run_state_vector(initial_state).reshape((2**N, 1))
    # calculate the hermitian conjuagte the output state
    fin_state_conj = paddle.conj(fin_state).transpose(perm=(1, 0))
    # comput the inner product between the two states
    inner_prod = paddle.matmul(fin_state_conj, target_state)
    # compute the square of the inner product, and use it as the loss
    loss = - paddle.real(paddle.multiply(inner_prod, paddle.conj(inner_prod)))

    return loss, cir

/Users/zihanxia/anaconda3/envs/paddle_quantum/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
class StateNet(paddle.nn.Layer):
    """
    Construct the model net
    """

    def __init__(self, shape, dtype="float64"):
        super(StateNet, self).__init__()
        
        # Initialize the list of theta parameters, filling the initial values with a uniform distribution of [0, 2* PI]  
        self.theta = self.create_parameter(shape=shape, 
                                           default_initializer=paddle.nn.initializer.Uniform(low=0.0, high=2*PI),
                                           dtype=dtype, is_bias=False)
        
   # Define loss function and forward propagation mechanism
    def forward(self, N, D):
        
        # get the randomly chosen four states |0000>,|0011>,|0111>,|1011>
        initial_state = []
        initial_state.append(paddle.to_tensor(np.eye(2**N)[0], 'complex128'))
        initial_state.append(paddle.to_tensor(np.eye(2**N)[4], 'complex128'))
        initial_state.append(paddle.to_tensor(np.eye(2**N)[8], 'complex128'))
        initial_state.append(paddle.to_tensor(np.eye(2**N)[12], 'complex128'))
        # construct the target four states |0011>,|0101>,|1010>,|1100>
        states = []
        states.append(paddle.to_tensor(np.eye(2**N)[2], 'float64'))
        states.append(paddle.to_tensor(np.eye(2**N)[4], 'float64'))
        states.append(paddle.to_tensor(np.eye(2**N)[9], 'float64'))
        states.append(paddle.to_tensor(np.eye(2**N)[11], 'float64'))
        # for each given initial state, calculate the loss, and add them together as the final loss
        final_loss, cir = U_theta(self.theta, N, D, initial_state[0], states[0])
        for i in range(3):
            loss, cir = U_theta(self.theta, N, D, initial_state[i+1], states[i+1])
            final_loss += loss

        return final_loss, cir

In [4]:
ITR = 120  # Set the number of optimization iterations
LR = 0.2   # Set the learning rate
D = 6      # Set the depth of the repetitive calculation module in QNN
N = 4 # number of qubits = 4

In [5]:
# Determine the parameter dimensions of the network 
net = StateNet(shape=[D + 1, N, 3])

# use the Adam optimizer to obtain relatively good convergence
opt = paddle.optimizer.Adam(learning_rate=LR, parameters=net.parameters())

# Optimize iterations
for itr in range(1, ITR + 1):

    # Forward propagation calculates the loss function
    loss, cir = net(N, D)

    # Back propagation minimizes the loss function
    loss.backward()
    opt.minimize(loss)
    opt.clear_grad()

    # Print results
    if itr % 20 == 0:
        print("iter:", itr, "loss:", "%.4f" % paddle.cast(loss, 'float64').numpy())
    if itr == ITR:
        print("\nCircuit after training:") 
        print(cir)


/Users/zihanxia/anaconda3/envs/paddle_quantum/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


iter: 20 loss: -3.7913
iter: 40 loss: -3.9635
iter: 60 loss: -3.9970
iter: 80 loss: -3.9993
iter: 100 loss: -3.9999
iter: 120 loss: -4.0000

Circuit after training:
--U----*--------------x----U----*--------------x----U----*--------------x----U----*--------------x----U----*--------------x----U----*--------------x----Ry(6.283)--
       |              |         |              |         |              |         |              |         |              |         |              |               
--U----x----*---------|----U----x----*---------|----U----x----*---------|----U----x----*---------|----U----x----*---------|----U----x----*---------|----Ry(1.570)--
            |         |              |         |              |         |              |         |              |         |              |         |               
--U---------x----*----|----U---------x----*----|----U---------x----*----|----U---------x----*----|----U---------x----*----|----U---------x----*----|----Ry(-0.00)--
               